In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import dagshub
import mlflow

In [33]:
dagshub.init(repo_owner='akshatsharma2407', repo_name='GMC_motors', mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/akshatsharma2407/GMC_motors.mlflow')

Accessing as akshatsharma2407

Initialized MLflow to track repo "akshatsharma2407/GMC_motors"

Repository akshatsharma2407/GMC_motors initialized!

In [34]:
mlflow.autolog()
mlflow.set_experiment(experiment_name='Random_forest_best_HP')
mlflow.start_run()

2025/03/10 19:52:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/10 19:52:25 INFO mlflow.tracking.fluent: Experiment with name 'Random_forest_best_HP' does not exist. Creating a new experiment.


<ActiveRun: >

In [35]:
data = pd.read_csv('C:/Users/aksha/OneDrive/Desktop/GMC_MLOPS/data/processed/train_processed_df.csv')

xtrain = data.drop(columns='Price($)')
ytrain = data['Price($)']

In [36]:
rf_hyperparams = {
    "n_estimators": [50, 100, 200, 500],  
    "max_depth": [None, 10, 20, 30, 50], 
    "min_samples_split": [2, 5, 10], 
    "min_samples_leaf": [1, 2, 4, 8], 
    "min_weight_fraction_leaf": [0.0, 0.1, 0.2],  
    "max_features": ["sqrt", "log2", None], 
    "max_leaf_nodes": [None, 10, 50, 100],  
    "min_impurity_decrease": [0.0, 0.01, 0.05], 
    "bootstrap": [True, False],  
    "ccp_alpha": [0.0, 0.01, 0.1]
}


In [37]:
model = RandomForestRegressor()

random_search = RandomizedSearchCV(model,param_distributions=rf_hyperparams,cv=2,verbose=1,scoring='r2')

random_search.fit(xtrain,ytrain)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


2025/03/10 19:53:31 INFO mlflow.sklearn.utils: Logging the 5 best runs, 5 runs will be omitted.


🏃 View run welcoming-deer-985 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5/runs/5f32dbda51d54650aa214a04625a1161
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5
🏃 View run honorable-sloth-187 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5/runs/cec453c75ba84a74a953a91eeb8e5241
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5
🏃 View run rogue-worm-10 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5/runs/7b57bb6feb3b4c04b997392f0599ed24
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5
🏃 View run big-squid-582 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5/runs/2a5f9fd305494a3993d5df5004af22e1
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5
🏃 View run enthused-bear-354 at: https://dagshub.com/akshatsharma

RandomizedSearchCV(cv=2, estimator=RandomForestRegressor(),
                   param_distributions={'bootstrap': [True, False],
                                        'ccp_alpha': [0.0, 0.01, 0.1],
                                        'max_depth': [None, 10, 20, 30, 50],
                                        'max_features': ['sqrt', 'log2', None],
                                        'max_leaf_nodes': [None, 10, 50, 100],
                                        'min_impurity_decrease': [0.0, 0.01,
                                                                  0.05],
                                        'min_samples_leaf': [1, 2, 4, 8],
                                        'min_samples_split': [2, 5, 10],
                                        'min_weight_fraction_leaf': [0.0, 0.1,
                                                                     0.2],
                                        'n_estimators': [50, 100, 200, 500]},
                   scoring='r2', verbose=1)

In [39]:
pd.DataFrame(random_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_weight_fraction_leaf,param_min_samples_split,param_min_samples_leaf,param_min_impurity_decrease,param_max_leaf_nodes,param_max_features,param_max_depth,param_ccp_alpha,param_bootstrap,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.799845,0.016379,0.049918,0.000948,200,0.2,5,8,0.05,50,None,10,0.01,False,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.701202,0.707613,0.704407,0.003206,10
1,0.420152,0.011793,0.071403,0.005519,200,0.1,10,8,0.00,100,log2,30,0.00,False,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.741237,0.755398,0.748318,0.007080,7
2,0.915818,0.026705,0.063154,0.010870,200,0.1,5,8,0.01,50,None,30,0.10,True,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.790125,0.790520,0.790323,0.000197,4
3,1.336620,0.276179,0.179188,0.011581,500,0.1,5,4,0.05,None,log2,20,0.10,False,"{'n_estimators': 500, 'min_weight_fraction_lea...",0.750084,0.746271,0.748178,0.001907,8
4,0.190470,0.000656,0.015349,0.000816,50,0.2,2,1,0.00,None,None,50,0.10,False,"{'n_estimators': 50, 'min_weight_fraction_leaf...",0.701202,0.707613,0.704407,0.003206,9
5,1.300591,0.014390,0.143296,0.003586,500,0.1,2,2,0.01,100,log2,20,0.01,True,"{'n_estimators': 500, 'min_weight_fraction_lea...",0.748573,0.749752,0.749162,0.000589,6
6,0.230559,0.023796,0.032443,0.001091,100,0.1,5,1,0.01,50,sqrt,None,0.01,False,"{'n_estimators': 100, 'min_weight_fraction_lea...",0.753703,0.760677,0.757190,0.003487,5
7,0.284462,0.003744,0.039585,0.000432,100,0.0,5,1,0.01,10,sqrt,50,0.01,False,"{'n_estimators': 100, 'min_weight_fraction_lea...",0.801966,0.805640,0.803803,0.001837,2
8,1.415007,0.013544,0.061590,0.005776,200,0.0,5,2,0.01,10,None,10,0.10,False,"{'n_estimators': 200, 'min_weight_fraction_lea...",0.825162,0.827868,0.826515,0.001353,1
9,2.313912,0.324691,0.217355,0.041905,500,0.1,10,4,0.00,10,None,30,0.10,True,"{'n_estimators': 500, 'min_weight_fraction_lea...",0.790939,0.789804,0.790372,0.000568,3


In [40]:
mlflow.end_run()

🏃 View run rogue-frog-786 at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5/runs/e6b75acceecc473caaf935addc7ce9d8
🧪 View experiment at: https://dagshub.com/akshatsharma2407/GMC_motors.mlflow/#/experiments/5
